In [1]:
#R version 3.6.3 (2020-02-29)
#loading library used in this work. Version were specified.
                     
library(fdapace)      #version 0.5.5
library(funData)      #version 1.3.5
library(MFPCA)        #version 1.3.6
library(raster)       #version 3.3.13
library(randomForest) #version 4.6.14
library(caret)        #version 6.0.86
library(rgdal)        #version 1.5.16

Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar


Attaching package: ‘funData’


The following object is masked from ‘package:stats’:

    integrate


Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:funData’:

    approxNA


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: lattice

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


The following object is masked from ‘package:funData’:

    ggplot


rgdal: version: 1.5-16, (SVN revision 1050)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.1.2, released 2020/07/07
Path to GDAL shared files: /usr/share/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.1.1, September 1st, 2020, [PJ_VERSION: 711]
Path to PROJ shared

In [2]:

# Define the file path of the shapefile
file.shape <- '/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Shapefile/TD_CONERO.shp'

# Read the shapefile into a spatial object
pu <- shapefile(file.shape)

# Define the file path of the smoothed time series
pathTS <- "/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Smoothed_Time_Series/F"


In [3]:
#loading function univariate FPCA, multivariate FPCA,
#Random Forest (including Random Feature Elimination) 
#combined in a Forward strategy

  source("/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Functions_R/FPCAuni.R")                  #uFPCA of a time series. it returns FPC scores
  source("/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Functions_R/RFforSelFunction.R")         #MFPCA of a multiple time series. it returns MFPC scores
  source("/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Functions_R/FPCAmulti.R")                #RF with RFE using FPC/MFPC scores as input data
  source("/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Functions_R/ForwardSelectionFunction.R") #it combines uFPCA/MFPCA with RF in forward selectin strategy


In [4]:
# Loop over the values in the vector 
for (var in c('N')) {
  # Assign the current value to the variable 'stand'
  stand <- var
  
  # Loop over the values in the vector "20"
  for (folder in c("01")) {
    # Concatenate the path of the smoothed time series and the current folder value
    path <- paste(pathTS, folder, sep='')
    # Set the working directory to the path
    setwd(path)
    # List all the files in the directory that end with '.txt' and convert them to a vector
    files <- as.vector(list.files(pattern = "\\.txt$"))
    
    # Apply the function 'forSelFunzione' to the stand, path, and files arguments and store the result in 'prodotti'
    prodotti <- forSelFunzione(stand, path, files)
    
    # Create a file name for the best time series by concatenating the folder, 'TSsel', and stand values, separated by underscores, and adding the '.txt' extension
    nome.bestTS <- paste(paste(folder, 'TSsel', stand, sep='_'), '.txt', sep='')
    
    # Write the 'TSsel' component of 'prodotti' to a file with the name 'nome.bestTS' in the 'Selected' subdirectory of the smoothed time series directory
    write.table(prodotti$TSsel, paste('/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Smoothed_Time_Series/Selected/F', nome.bestTS, sep=''))
    # Write the 'Top' component of 'prodotti' to a file with the name 'folder_TopResult_stand.txt' in the 'Accuracy' subdirectory of the 'Selected' subdirectory
    write.table(prodotti$Top, paste('/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Smoothed_Time_Series/Selected/Accuracy/F', paste(paste(folder, 'TopResult', stand, sep='_'), '.txt', sep=''), sep=''))
    # Write the 'All' component of 'prodotti' to a file with the name 'folder_AllResults_stand.txt' in the 'Accuracy' subdirectory of the 'Selected' subdirectory
    write.table(prodotti$All, paste('/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Smoothed_Time_Series/Selected/Accuracy/F', paste(paste(folder, 'AllResults', stand, sep='_'), '.txt', sep=''), sep=''))
  }
}


[1] "F01_03_02_00_00_10_BIL.txt (univariate FPCA, RF with RFE of FPCA scores)"
  mtry Accuracy     Kappa AccuracySD   KappaSD                         TS
1    1 0.786726 0.6769767 0.08163024 0.1264468 F01_03_02_00_00_10_BIL.txt
[1] "F01_06_02_00_00_10_BIL.txt (univariate FPCA, RF with RFE of FPCA scores)"
  mtry  Accuracy     Kappa AccuracySD  KappaSD                         TS
1    1 0.8267798 0.7470286 0.08599664 0.125625 F01_06_02_00_00_10_BIL.txt
[1] "F01_07_02_00_00_10_BIL.txt (univariate FPCA, RF with RFE of FPCA scores)"
  mtry  Accuracy     Kappa AccuracySD   KappaSD                         TS
3    3 0.7997708 0.7025775 0.07769677 0.1169242 F01_07_02_00_00_10_BIL.txt
[1] "*****seconda fase multiFPCA*****"
[1] "F01_06_02_00_00_10_BIL.txt (multivariate FPCA, RF with RFE of FPCA scores)"
[2] "F01_03_02_00_00_10_BIL.txt (multivariate FPCA, RF with RFE of FPCA scores)"


Warning message in MFPCA(m.funzioni, M = 36, approx.eigen = FALSE, uniExpansions = lista.exp, :
“Function MFPCA: total number of univariate basis functions is smaller than given M. M was set to 21.”


  mtry  Accuracy     Kappa AccuracySD   KappaSD
2    2 0.7990209 0.6984542 0.09783886 0.1495305
                                                     TS
2 F01_06_02_00_00_10_BIL.txt F01_03_02_00_00_10_BIL.txt
[1] "F01_06_02_00_00_10_BIL.txt (multivariate FPCA, RF with RFE of FPCA scores)"
[2] "F01_07_02_00_00_10_BIL.txt (multivariate FPCA, RF with RFE of FPCA scores)"


Warning message in MFPCA(m.funzioni, M = 36, approx.eigen = FALSE, uniExpansions = lista.exp, :
“Function MFPCA: total number of univariate basis functions is smaller than given M. M was set to 20.”


  mtry  Accuracy     Kappa AccuracySD   KappaSD
1    1 0.8349862 0.7512827 0.07483358 0.1158457
                                                     TS
1 F01_06_02_00_00_10_BIL.txt F01_07_02_00_00_10_BIL.txt
[1] "F01_06_02_00_00_10_BIL.txt (multivariate FPCA, RF with RFE of FPCA scores)"
[2] "F01_07_02_00_00_10_BIL.txt (multivariate FPCA, RF with RFE of FPCA scores)"
[3] "F01_03_02_00_00_10_BIL.txt (multivariate FPCA, RF with RFE of FPCA scores)"


Warning message in MFPCA(m.funzioni, M = 36, approx.eigen = FALSE, uniExpansions = lista.exp, :
“Function MFPCA: total number of univariate basis functions is smaller than given M. M was set to 31.”


  mtry  Accuracy     Kappa AccuracySD   KappaSD
1    1 0.8267565 0.7394682 0.07496604 0.1161347
                                                                                TS
1 F01_06_02_00_00_10_BIL.txt F01_07_02_00_00_10_BIL.txt F01_03_02_00_00_10_BIL.txt
[1] "procedura terminata"
